In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier


train = pd.read_csv("./train.csv", index_col= 0)
test = pd.read_csv("./test.csv", index_col= 0)
submission = pd.read_csv("./submission.csv" , index_col= 0)


In [2]:
train.corr()

C:\Users\이나라\AppData\Local\Temp\ipykernel_20656\2189804198.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.corr()


,Survived,Pclass,Age,SibSp,Parch,Fare
Survived,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Name      418 non-null    object 
 2   Sex       418 non-null    object 
 3   Age       332 non-null    float64
 4   SibSp     418 non-null    int64  
 5   Parch     418 non-null    int64  
 6   Ticket    418 non-null    object 
 7   Fare      417 non-null    float64
 8   Cabin     91 non-null     object 
 9   Embarked  418 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 35.9+ KB


# AGE....

In [5]:
a = train['Name'].str.contains('Miss\.')
# b = train['Name'].str.contains('Mr\.')
c = train['Name'].str.contains('Mrs\.')
d = train['Name'].str.contains('Master\.')
e = train['Name'].str.contains('Dr\.')

miss_age = train[a]['Age'].mean().__round__(1)
# mr_age = train[b]['Age'].mean().__round__(1)
mrs_age = train[c]['Age'].mean().__round__(1)
master_age = train[d]['Age'].mean().__round__(1)
dr_age = train[e]['Age'].mean().__round__(1)

# mr_t1 = train.loc[(train['Age'] >= 30) & train['Name'].str.contains('Mr\.') ]
# mr_t2 = train.loc[(train['Age'] < 30) & train['Name'].str.contains('Mr\.') ]

mr_t3 = train.loc[(train['Fare'] >= 10.5) & train['Name'].str.contains('Mr\.') & train['Age'].notnull()]
mr_t4 = train.loc[(train['Fare'] < 10.5) & train['Name'].str.contains('Mr\.') & train['Age'].notnull()]

mr_t5 =  mr_t3['Age'].mean().__round__(1)
mr_t6 =  mr_t4['Age'].mean().__round__(1)

# train.loc[train['Name'].str.contains('Mr\.'), 'Age'] = train[train['Name'].str.contains('Mr\.')]['Age'].fillna(mr_age)

train.loc[train['Name'].str.contains('Miss\.'), 'Age'] = train[train['Name'].str.contains('Miss\.')]['Age'].fillna(miss_age)
train.loc[train['Name'].str.contains('Mrs\.'), 'Age'] = train[train['Name'].str.contains('Mrs\.')]['Age'].fillna(mrs_age)
train.loc[train['Name'].str.contains('Master\.'), 'Age'] = train[train['Name'].str.contains('Master\.')]['Age'].fillna(master_age)
train.loc[(train['Name'].str.contains('Mr\.')) & train['Fare'] >= 10.5 , 'Age'] = train.loc[(train['Name'].str.contains('Mr\.')) & train['Fare'] >= 10.5 , 'Age'].fillna(mr_t5)
train.loc[(train['Name'].str.contains('Mr\.')) & train['Fare'] < 10.5 , 'Age'] = train.loc[(train['Name'].str.contains('Mr\.')) & train['Fare'] < 10.5 , 'Age'].fillna(mr_t6)
train.loc[train['Name'].str.contains('Dr\.'), 'Age'] = train[train['Name'].str.contains('Dr\.')]['Age'].fillna(mr_t5)

# test.loc[test['Name'].str.contains('Mr\.'), 'Age'] = test[test['Name'].str.contains('Mr\.')]['Age'].fillna(mr_age)

test.loc[test['Name'].str.contains('Miss\.'), 'Age'] = test[test['Name'].str.contains('Miss\.')]['Age'].fillna(miss_age)
test.loc[test['Name'].str.contains('Mrs\.'), 'Age'] = test[test['Name'].str.contains('Mrs\.')]['Age'].fillna(mrs_age)
test.loc[test['Name'].str.contains('Master\.'), 'Age'] = test[test['Name'].str.contains('Master\.')]['Age'].fillna(master_age)
test.loc[(test['Name'].str.contains('Mr\.')) & test['Fare'] >= 10.5 , 'Age'] = test.loc[(test['Name'].str.contains('Mr\.')) & test['Fare'] >= 10.5 , 'Age'].fillna(mr_t5)
test.loc[(test['Name'].str.contains('Mr\.')) & test['Fare'] < 10.5 , 'Age'] = test.loc[(test['Name'].str.contains('Mr\.')) & test['Fare'] < 10.5 , 'Age'].fillna(mr_t6)
test.loc[test['Name'].str.contains('Dr\.'), 'Age'] = test[test['Name'].str.contains('Dr\.')]['Age'].fillna(mr_t5)
test.loc[test['Name'].str.contains('Ms\.'), 'Age'] = test[test['Name'].str.contains('Ms\.')]['Age'].fillna(miss_age)

In [6]:
print(train['Age'].isnull().sum(), test['Age'].isnull().sum())

0 0


# 처리

In [7]:
train['Sex'] = train['Sex'].replace('male',0)
test['Sex'] = test['Sex'].replace('male',0)
train['Sex'] = train['Sex'].replace('female',1)
test['Sex'] = test['Sex'].replace('female',1)

In [8]:
train['Embarked'] = train['Embarked'].fillna("S")
test['Embarked'] = test['Embarked'].fillna("S")

train = pd.get_dummies(train, columns=['Embarked'])
test = pd.get_dummies(test, columns=['Embarked'])

In [9]:
test

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,
892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,0,1,0
893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,0,0,1
894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,0,1,0
895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,0,0,1
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",0,29.1,0,0,A.5. 3236,8.0500,NaN,0,0,1
1306,1,"Oliva y Ocana, Dona. Fermina",1,39.0,0,0,PC 17758,108.9000,C105,1,0,0
1307,3,"Saether, Mr. Simon Sivertsen",0,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,0,0,1


In [10]:
test['Fare'] = test['Fare'].fillna((train.loc[train['Pclass'] == 3])['Fare'].mean())

In [11]:
train['Companion'] = train['SibSp'] + train['Parch']

train['TicketPer1'] = train['Fare'] / (train['Companion'] + 1)

test['Companion'] = test['SibSp'] + test['Parch']

test['TicketPer1'] = test['Fare'] / (test['Companion'] + 1)

In [12]:
test

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S,Companion,TicketPer1
PassengerId,,,,,,,,,,,,,,
892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,0,1,0,0,7.829200
893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,0,0,1,1,3.500000
894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,0,1,0,0,9.687500
895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,0,0,1,0,8.662500
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,0,0,1,2,4.095833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",0,29.1,0,0,A.5. 3236,8.0500,NaN,0,0,1,0,8.050000
1306,1,"Oliva y Ocana, Dona. Fermina",1,39.0,0,0,PC 17758,108.9000,C105,1,0,0,0,108.900000
1307,3,"Saether, Mr. Simon Sivertsen",0,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,0,0,1,0,7.250000


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Name        891 non-null    object 
 3   Sex         891 non-null    int64  
 4   Age         891 non-null    float64
 5   SibSp       891 non-null    int64  
 6   Parch       891 non-null    int64  
 7   Ticket      891 non-null    object 
 8   Fare        891 non-null    float64
 9   Cabin       204 non-null    object 
 10  Embarked_C  891 non-null    uint8  
 11  Embarked_Q  891 non-null    uint8  
 12  Embarked_S  891 non-null    uint8  
 13  Companion   891 non-null    int64  
 14  TicketPer1  891 non-null    float64
dtypes: float64(3), int64(6), object(3), uint8(3)
memory usage: 93.1+ KB


In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      418 non-null    int64  
 1   Name        418 non-null    object 
 2   Sex         418 non-null    int64  
 3   Age         418 non-null    float64
 4   SibSp       418 non-null    int64  
 5   Parch       418 non-null    int64  
 6   Ticket      418 non-null    object 
 7   Fare        418 non-null    float64
 8   Cabin       91 non-null     object 
 9   Embarked_C  418 non-null    uint8  
 10  Embarked_Q  418 non-null    uint8  
 11  Embarked_S  418 non-null    uint8  
 12  Companion   418 non-null    int64  
 13  TicketPer1  418 non-null    float64
dtypes: float64(3), int64(5), object(3), uint8(3)
memory usage: 40.4+ KB


# CABIN

In [15]:
cabin_train = train.loc[train['Cabin'].notnull()]
cabin_test = train.loc[train['Cabin'].isnull()]

testC = test.loc[test['Cabin'].isnull()]

cabin_train['Ticketnum'] = cabin_train['Ticket'].apply(lambda x: ''.join(re.findall(r'\d+', x)))

cabin_test['Ticketnum'] = cabin_test['Ticket'].apply(lambda x: ''.join(re.findall(r'\d+', x)))

testC['Ticketnum'] = testC['Ticket'].apply(lambda x: ''.join(re.findall(r'\d+', x)))


C:\Users\이나라\AppData\Local\Temp\ipykernel_20656\3795220210.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cabin_train['Ticketnum'] = cabin_train['Ticket'].apply(lambda x: ''.join(re.findall(r'\d+', x)))
C:\Users\이나라\AppData\Local\Temp\ipykernel_20656\3795220210.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cabin_test['Ticketnum'] = cabin_test['Ticket'].apply(lambda x: ''.join(re.findall(r'\d+', x)))
C:\Users\이나라\AppData\Local\Temp\ipykernel_20656\3795220210.py:10: SettingWithCopyWarning: 
A value

In [16]:
cabin_data = cabin_train[[ 'Pclass',  'SibSp', 'Parch', 'Embarked_C','Embarked_S','Embarked_Q', 'Fare', 'TicketPer1', 'Ticketnum' , 'Cabin']]
cabin_data_test = cabin_test[[ 'Pclass',  'SibSp', 'Parch', 'Embarked_C', 'Embarked_S', 'Embarked_Q', 'Fare', 'TicketPer1', 'Ticketnum' ]]
cabin_data_test1 = cabin_data_test.drop([180, 272, 303, 598])
cabin_data_test['Ticketnum'] = cabin_data_test['Ticketnum'].replace('', cabin_data_test1['Ticketnum'].median())

test_data = testC[[ 'Pclass',  'SibSp', 'Parch', 'Embarked_C', 'Embarked_S', 'Embarked_Q', 'Fare', 'TicketPer1', 'Ticketnum' ]]
test_data['Ticketnum'] = test_data['Ticketnum'].replace('', cabin_data_test1['Ticketnum'].median())
 
cabin_X = cabin_data.iloc[:,:-1].values
cabin_y = cabin_data.iloc[:,-1].values

len(cabin_X) , len(cabin_y)

C:\Users\이나라\AppData\Local\Temp\ipykernel_20656\3642274198.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cabin_data_test['Ticketnum'] = cabin_data_test['Ticketnum'].replace('', cabin_data_test1['Ticketnum'].median())
C:\Users\이나라\AppData\Local\Temp\ipykernel_20656\3642274198.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Ticketnum'] = test_data['Ticketnum'].replace('', cabin_data_test1['Ticketnum'].median())


(204, 204)

In [17]:
cabin_data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 687 entries, 1 to 891
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      687 non-null    int64  
 1   SibSp       687 non-null    int64  
 2   Parch       687 non-null    int64  
 3   Embarked_C  687 non-null    uint8  
 4   Embarked_S  687 non-null    uint8  
 5   Embarked_Q  687 non-null    uint8  
 6   Fare        687 non-null    float64
 7   TicketPer1  687 non-null    float64
 8   Ticketnum   687 non-null    object 
dtypes: float64(2), int64(3), object(1), uint8(3)
memory usage: 55.7+ KB


In [18]:
cabin_X_train, cabin_X_test, cabin_y_train, cabin_y_test = train_test_split(cabin_X, cabin_y, test_size = 0.2, random_state = 0)

random_forest = RandomForestClassifier(n_estimators=500)
random_forest.fit(cabin_X_train, cabin_y_train)
Y_pred = random_forest.predict(cabin_X_test)

random_forest.score(cabin_X_train, cabin_y_train)

0.9754601226993865

In [19]:
cabin_y_pred_test = random_forest.predict(cabin_data_test)

test_cabin_pred = random_forest.predict(test_data)

c:\Anaconda3\envs\study\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
c:\Anaconda3\envs\study\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [20]:
cabin_y_pred_test

array(['F G73', 'E10', 'E10', 'F38', 'G6', 'G6', 'F33', 'E10', 'G6',
       'F G73', 'F33', 'G6', 'F33', 'E10', 'D', 'F33', 'F38', 'G6', 'G6',
       'D', 'F38', 'F G73', 'B4', 'F38', 'F33', 'D33', 'C123', 'D', 'E10',
       'G6', 'G6', 'G6', 'E101', 'F38', 'D', 'F38', 'E10', 'F38', 'F38',
       'F E69', 'E10', 'F4', 'F G73', 'D', 'F33', 'D', 'F2', 'G6', 'D',
       'G6', 'B4', 'G6', 'E77', 'G6', 'G6', 'E101', 'G6', 'E101', 'D',
       'D', 'F G73', 'E10', 'F2', 'F33', 'E10', 'F33', 'F38', 'A32',
       'F33', 'G6', 'G6', 'E10', 'E10', 'E10', 'F G63', 'G6', 'F G73',
       'E10', 'F2', 'F33', 'F G73', 'F G73', 'E77', 'G6', 'F G73',
       'F G63', 'F G63', 'F G73', 'F38', 'D', 'E10', 'G6', 'D', 'E10',
       'F38', 'E101', 'G6', 'C101', 'E10', 'F33', 'G6', 'F38', 'F G63',
       'F G63', 'F38', 'F G73', 'F33', 'F2', 'D', 'D', 'E77', 'G6',
       'F G63', 'E10', 'F G73', 'F33', 'F4', 'F G73', 'F4', 'F33', 'D',
       'E10', 'G6', 'F G63', 'D10 D12', 'F38', 'E10', 'E10', 'G6', 'E121',
 

In [21]:
cabin_test['Cabin'] = cabin_y_pred_test

train['Cabin'] = train['Cabin'].fillna(cabin_test['Cabin'])

C:\Users\이나라\AppData\Local\Temp\ipykernel_20656\2962975947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cabin_test['Cabin'] = cabin_y_pred_test


In [22]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Name        891 non-null    object 
 3   Sex         891 non-null    int64  
 4   Age         891 non-null    float64
 5   SibSp       891 non-null    int64  
 6   Parch       891 non-null    int64  
 7   Ticket      891 non-null    object 
 8   Fare        891 non-null    float64
 9   Cabin       891 non-null    object 
 10  Embarked_C  891 non-null    uint8  
 11  Embarked_Q  891 non-null    uint8  
 12  Embarked_S  891 non-null    uint8  
 13  Companion   891 non-null    int64  
 14  TicketPer1  891 non-null    float64
dtypes: float64(3), int64(6), object(3), uint8(3)
memory usage: 93.1+ KB


In [23]:
testC['Cabin'] = test_cabin_pred

test['Cabin'] = test['Cabin'].fillna(testC['Cabin'])



C:\Users\이나라\AppData\Local\Temp\ipykernel_20656\3166777446.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testC['Cabin'] = test_cabin_pred


In [24]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      418 non-null    int64  
 1   Name        418 non-null    object 
 2   Sex         418 non-null    int64  
 3   Age         418 non-null    float64
 4   SibSp       418 non-null    int64  
 5   Parch       418 non-null    int64  
 6   Ticket      418 non-null    object 
 7   Fare        418 non-null    float64
 8   Cabin       418 non-null    object 
 9   Embarked_C  418 non-null    uint8  
 10  Embarked_Q  418 non-null    uint8  
 11  Embarked_S  418 non-null    uint8  
 12  Companion   418 non-null    int64  
 13  TicketPer1  418 non-null    float64
dtypes: float64(3), int64(5), object(3), uint8(3)
memory usage: 40.4+ KB


In [25]:
train['Cabin_G'] = train['Cabin'].apply(lambda x: ''.join(re.findall(r'^\w', x)))

test['Cabin_G'] = test['Cabin'].apply(lambda x: ''.join(re.findall(r'^\w', x)))

train['Cabin_G'] = train['Cabin_G'].replace('A',7)
train['Cabin_G'] = train['Cabin_G'].replace('B',6)
train['Cabin_G'] = train['Cabin_G'].replace('C',5)
train['Cabin_G'] = train['Cabin_G'].replace('D',4)
train['Cabin_G'] = train['Cabin_G'].replace('E',3)
train['Cabin_G'] = train['Cabin_G'].replace('F',2)
train['Cabin_G'] = train['Cabin_G'].replace('G',1)
train['Cabin_G'] = train['Cabin_G'].replace('T',6)

test['Cabin_G'] = test['Cabin_G'].replace('A',7)
test['Cabin_G'] = test['Cabin_G'].replace('B',6)
test['Cabin_G'] = test['Cabin_G'].replace('C',5)
test['Cabin_G'] = test['Cabin_G'].replace('D',4)
test['Cabin_G'] = test['Cabin_G'].replace('E',3)
test['Cabin_G'] = test['Cabin_G'].replace('F',2)
test['Cabin_G'] = test['Cabin_G'].replace('G',1)


In [26]:
train.corr()

C:\Users\이나라\AppData\Local\Temp\ipykernel_20656\2189804198.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.corr()


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Companion,TicketPer1,Cabin_G
Survived,1.000000,-0.338481,0.543351,-0.070478,-0.035322,0.081629,0.257307,0.168240,0.003650,-0.149683,0.016639,0.221600,0.193076
Pclass,-0.338481,1.000000,-0.131900,-0.354059,0.083081,0.018443,-0.549500,-0.243292,0.221009,0.074053,0.065997,-0.485079,-0.722413
Sex,0.543351,-0.131900,1.000000,-0.092505,0.114631,0.245489,0.182333,0.082853,0.074115,-0.119224,0.200988,0.115143,-0.010000
Age,-0.070478,-0.354059,-0.092505,1.000000,-0.261424,-0.184093,0.100303,0.038799,-0.061980,0.005015,-0.270644,0.150916,0.319874
SibSp,-0.035322,0.083081,0.114631,-0.261424,1.000000,0.414838,0.159651,-0.059528,-0.026354,0.068734,0.890712,-0.094682,-0.243948
Parch,0.081629,0.018443,0.245489,-0.184093,0.414838,1.000000,0.216225,-0.011069,-0.081228,0.060814,0.783111,-0.068978,-0.204678
Fare,0.257307,-0.549500,0.182333,0.100303,0.159651,0.216225,1.000000,0.269335,-0.117216,-0.162184,0.217138,0.840995,0.457178
Embarked_C,0.168240,-0.243292,0.082853,0.038799,-0.059528,-0.011069,0.269335,1.000000,-0.148258,-0.782742,-0.046215,0.271215,0.373886
Embarked_Q,0.003650,0.221009,0.074115,-0.061980,-0.026354,-0.081228,-0.117216,-0.148258,1.000000,-0.499421,-0.058592,-0.096038,-0.200331
Embarked_S,-0.149683,0.074053,-0.119224,0.005015,0.068734,0.060814,-0.162184,-0.782742,-0.499421,1.000000,0.077359,-0.177159,-0.201471


train_r = train[['Pclass', 'Sex', 'Cabin_G', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Companion', 'TicketPer1', 'Survived' ]]
test_r = test[['Pclass', 'Sex', 'Cabin_G', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Companion', 'TicketPer1']]

X = train_r.iloc[:,:-1].values
y = train_r.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [27]:
ss = StandardScaler()

ss.fit(train[['TicketPer1']])
train_fare = ss.transform(train[['TicketPer1']])
test_fare = ss.transform(test[['TicketPer1']])

train['TicketPer1'] = train_fare
test['TicketPer1'] = test_fare



In [28]:
train_r = train[['Pclass', 'Sex','Age', 'Cabin_G', 'Companion', 'TicketPer1','Fare', 'Survived' ]]
test_r = test[['Pclass', 'Sex', 'Age','Cabin_G', 'Companion', 'TicketPer1','Fare']]


X = train_r.iloc[:,:-1].values
y = train_r.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [29]:
test_r

,Pclass,Sex,Age,Cabin_G,Companion,TicketPer1,Fare
PassengerId,,,,,,,
892,3,0,34.5,2,0,-0.337431,7.8292
893,3,1,47.0,2,1,-0.458287,7.0000
894,2,0,62.0,2,0,-0.285554,9.6875
895,3,0,27.0,3,0,-0.314169,8.6625
896,3,1,22.0,1,2,-0.441654,12.2875
...,...,...,...,...,...,...,...
1305,3,0,29.1,3,0,-0.331267,8.0500
1306,1,1,39.0,5,0,2.484109,108.9000
1307,3,0,38.5,2,0,-0.353601,7.2500


## randomforest

In [30]:
# random_forest.fit(X_train, y_train)

# random_forest.score(X_train, y_train)

# y_pred_test = random_forest.predict(test_r)

## regreesion

In [31]:
# regressor = LogisticRegression()

# regressor.fit(X_train, y_train)

# regressor.score(X_train, y_train)

# y_pred_test = regressor.predict(test_r)

## gradientBoosting

In [32]:
gbc = GradientBoostingClassifier(max_depth=1, random_state=0)

gbc.fit(X_train, y_train)

accuracy = gbc.score(X_test, y_test)

y_pred_test = gbc.predict(test_r)

print(f"Accuracy: {accuracy}")

Accuracy: 0.8212290502793296


c:\Anaconda3\envs\study\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [33]:
submission['Survived']  = y_pred_test

In [34]:
submission.to_csv('submission.csv', index=True)